If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [99]:
! pip install datasets transformers seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

Then you need to install Git-LFS. Uncomment the following instructions:

In [100]:
 !apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [101]:
import transformers

print(transformers.__version__)

4.28.1


You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/token-classification).

We also quickly upload some telemetry - this tells us which examples and software versions are getting used so we know where to prioritize our maintenance efforts. We don't collect (or care about) any personally identifiable information, but if you'd prefer not to be counted, feel free to skip this step or delete this cell entirely.

In [102]:
from transformers.utils import send_example_telemetry

send_example_telemetry("token_classification_notebook", framework="pytorch")

# Fine-tuning a model on a token classification task

This notebook is built to run on any token classification task, with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a version with a token classification head and a fast tokenizer (check on [this table](https://huggingface.co/transformers/index.html#bigtable) if this is the case). It might just need some small adjustments if you decide to use a different dataset than the one used here. Depending on you model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set those three parameters, then the rest of the notebook should run smoothly:

In [103]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "dslim/bert-base-NER"
batch_size = 40

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.  

In [104]:
from datasets import load_dataset, load_metric

For our example here, we'll use the [CONLL 2003 dataset](https://www.aclweb.org/anthology/W03-0419.pdf). The notebook should work with any token classification dataset provided by the 🤗 Datasets library. If you're using your own dataset defined from a JSON or csv file (see the [Datasets documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) on how to load them), it might need some adjustments in the names of the columns used.

In [105]:
datasets = load_dataset("conll2003")

  0%|          | 0/3 [00:00<?, ?it/s]

The `datasets` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set.

In [106]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

We can see the training, validation and test sets all have a column for the tokens (the input texts split into words) and one column of labels for each kind of task we introduced before.

To access an actual element, you need to select a split first, then give an index:

The labels are already coded as integer ids to be easily usable by our model, but the correspondence with the actual categories is stored in the `features` of the dataset:

In [107]:
datasets["train"].features[f"ner_tags"]

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

So for the NER tags, 0 corresponds to 'O', 1 to 'B-PER' etc... On top of the 'O' (which means no special entity), there are four labels for NER here, each prefixed with 'B-' (for beginning) or 'I-' (for intermediate), that indicate if the token is the first one for the current group with the label or not:
- 'PER' for person
- 'ORG' for organization
- 'LOC' for location
- 'MISC' for miscellaneous

Since the labels are lists of `ClassLabel`, the actual names of the labels are nested in the `feature` attribute of the object above:

In [108]:
label_list = datasets["train"].features[f"{task}_tags"].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [109]:
!wget https://raw.githubusercontent.com/zhangbo2008/data_ner/main/aomanyupianjian -O aaaa
!pip install datasets

--2023-05-05 02:01:22--  https://raw.githubusercontent.com/zhangbo2008/data_ner/main/aomanyupianjian
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44569 (44K) [text/plain]
Saving to: ‘aaaa’

aaaa                100%[===================>]  43.52K  --.-KB/s    in 0.001s  

2023-05-05 02:01:22 (42.4 MB/s) - ‘aaaa’ saved [44569/44569]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [110]:
#=======加载自己数据集

with open('aaaa') as f:
    t=f.readlines()
save1=[]
save2=[]
a=''
b=''
for i in t:

    if i=='\n':
        save1.append(a[:-1])
        save2.append(b[:-1])
        a=''
        b=''
    else:
        a+=i.split(' ')[0]+' '
        b+=i.split(' ')[1].replace('\n',' ')
print(3)
all2=[]
import copy


aaa=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']
tmp={'tokens':[], 'ner_tags':[]}
for i,j in zip(save1,save2):
  if i:

    tmp=tmp.copy()
    # tmp=copy.deepcopy(tmp) #没有多级数组所以可以用浅拷贝.
    tmp['tokens']=i.split(' ')
    tmp['ner_tags']=j.split(' ')#末尾句号一般也要进行独立设置O
    if '.' in tmp['tokens'][-1] and '.' !=tmp['tokens'][-1]:
        tmp['tokens'][-1]=tmp['tokens'][-1][:-1]
        tmp['tokens'].append('.')
        tmp['ner_tags'].append('O')
    if ',' in tmp['tokens'][-1] and ',' !=tmp['tokens'][-1]:
        tmp['tokens'][-1]=tmp['tokens'][-1][:-1]
        tmp['tokens'].append(',')
        tmp['ner_tags'].append('O')

    for kkk in range(len(tmp['ner_tags'])):
         tmp['ner_tags'][kkk]=tmp['ner_tags'][kkk].replace('B-Person','B-PER').replace('I-Person','I-PER').replace('B-Location','B-LOC').replace('I-Location','I-LOC').replace('B-Misc','B-MISC').replace('I-Misc','I-MISC')
    for i1 in range(len(tmp['ner_tags'])):
        for j1 in range(len(aaa)):
            if tmp['ner_tags'][i1]==aaa[j1]:
                tmp['ner_tags'][i1]=j1
    all2.append(tmp)
    print(111)


from datasets import Dataset
ds = Dataset.from_list(all2)
#===========tag进行编码
#'O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'






ds[0]
ds2=ds.train_test_split(test_size=0.3)#下面我们使用ds2即可.
ds2

3
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
111
11

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 307
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 132
    })
})

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset (automatically decoding the labels in passing).

In [111]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML
datasets=ds2
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [112]:
show_random_elements(datasets["train"])

,tokens,ner_tags
0,"[and,, though, she, answered, mechanically, to, the, repeated, appeals, of, her, uncle, and, aunt, ,]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,"[encouraged, her, communicativeness, by, his, questions, and, remarks;]","[0, 0, 0, 0, 0, 0, 0, 0]"
2,"[and, as, she, named, their, relationship, to, herself,, she, stole, a, sly, look, at, him, ,]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,"[The, occurrences, of, the, day, were, too, full, of, interest, to, leave, Elizabeth, much, attention, for, any, of, these, new, friends;, and, she, could, do, nothing, but, think,, and, think, with, wonder,, of, Mr., Darcy, 's, civility,, and, above, all,, of, his, wishing, her, to, be, acquainted, with, his, sister, .]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,"[than, she, had, any, notion, of, finding, her, .]","[0, 0, 0, 0, 0, 0, 0, 0, 0]"
5,"[But, to, be, sure,, the, good, lady, who, shewed, us, the, house, did, give, him, a, most]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0]"
6,"[they, were, all, pressed, to, go, into, the, house, and, take, some, refreshment;]","[0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0]"
7,"[""What, can, be, the, meaning, of, that, emphatic, exclamation?"", cried, he, .]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
8,"[He, takes, them, now, for, people, of, fashion., .]","[0, 0, 0, 0, 0, 0, 0, 0, 0]"
9,"[""Yes,, very, handsome., .]","[0, 0, 0, 0]"


## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [113]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

The following assertion ensures that our tokenizer is a fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, and we will need some of the special features they have for our preprocessing.

In [114]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

You can check which type of models have a fast tokenizer available and which don't on the [big table of models](https://huggingface.co/transformers/index.html#bigtable).

You can directly call this tokenizer on one sentence:

In [115]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [101, 8667, 117, 1142, 1110, 1141, 5650, 106, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

If, as is the case here, your inputs have already been split into words, you should pass the list of words to your tokenzier with the argument `is_split_into_words=True`:

In [116]:
tokenizer(["Hello", ",", "this", "is", "one", "sentence", "split", "into", "words", "."], is_split_into_words=True)

{'input_ids': [101, 8667, 117, 1142, 1110, 1141, 5650, 3325, 1154, 1734, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Note that transformers are often pretrained with subword tokenizers, meaning that even if your inputs have been split into words already, each of those words could be split again by the tokenizer. Let's look at an example of that:

In [117]:
example = datasets["train"][4]
print(example["tokens"])

['That', 'he', 'was', 'surprised', 'by', 'the', 'connexion', 'was', 'evident;']


In [118]:
11111111111

11111111111

In [119]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(222222222222)
33333333333
datasets

222222222222


DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 307
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 132
    })
})

In [120]:
#2023-05-04,21点21  下面我们尝试datasets进行数据替换


Here the words "Zwingmann" and "sheepmeat" have been split in three subtokens.

This means that we need to do some processing on our labels as the input ids returned by the tokenizer are longer than the lists of labels our dataset contain, first because some special tokens might be added (we can a `[CLS]` and a `[SEP]` above) and then because of those possible splits of words in multiple tokens:

In [121]:
len(example[f"{task}_tags"]), len(tokenized_input["input_ids"])

(9, 14)

Thankfully, the tokenizer returns outputs that have a `word_ids` method which can help us.

In [122]:
print(tokenized_input.word_ids())  #使用这个word_ids可以帮我们对齐token 和tag

[None, 0, 1, 2, 3, 4, 5, 6, 6, 6, 7, 8, 8, None]


As we can see, it returns a list with the same number of elements as our processed input ids, mapping special tokens to `None` and all other tokens to their respective word. This way, we can align the labels with the processed input ids.

In [123]:
word_ids = tokenized_input.word_ids() #=======这个数列的值表示.每一个token所对应的word的索引.
print(word_ids)
print(len(example[f"{task}_tags"]))
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids] #返回的数组是对齐之后的, 里面每一个token对应的tag都设置为了他所对应单词的tag!!!!!!!!
print(aligned_labels)
print(len(aligned_labels), len(tokenized_input["input_ids"])) # 我们把特殊符号都设置为-100, 这个-100会在计算loss时候自动去除.

[None, 0, 1, 2, 3, 4, 5, 6, 6, 6, 7, 8, 8, None]
9
[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]
14 14


Here we set the labels of all special tokens to -100 (the index that is ignored by PyTorch) and the labels of all other tokens to the label of the word they come from. Another strategy is to set the label only on the first token obtained from a given word, and give a label of -100 to the other subtokens from the same word. We propose the two strategies here, just change the value of the following flag:

In [124]:
label_all_tokens = True

We're now ready to write the function that will preprocess our samples. We feed them to the `tokenizer` with the argument `truncation=True` (to truncate texts that are bigger than the maximum size allowed by the model) and `is_split_into_words=True` (as seen above). Then we align the labels with the token ids using the strategy we picked:

In [125]:
def tokenize_and_align_labels(examples):#============这个函数吧上面的align写成函数.
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)# 得到第i个句子的 tokens 对应的word_id
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.                                                  
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:#这行代码的意思是, 如果当前token是一个word的开始部分.那么我们就加入他的label.
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:         #如果这个单词是一个后缀部分.那么我们可以考虑是否学习这个后缀.我理解还是学习他比较好一点. 这套代码默认也是学习这个.
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [126]:
datasets['train'][:1]

{'tokens': [['she',
   'added,',
   '"informed',
   'us',
   'that',
   'you',
   'would',
   'certainly',
   'not',
   'be',
   'here',
   'till',
   'to-morrow;']],
 'ner_tags': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]}

In [127]:
tokenize_and_align_labels(datasets['train'][:5])

{'input_ids': [[101, 1131, 1896, 117, 107, 6189, 1366, 1115, 1128, 1156, 4664, 1136, 1129, 1303, 6174, 1106, 118, 182, 1766, 7596, 132, 102], [101, 107, 1599, 117, 1139, 7059, 117, 1128, 1336, 1138, 1103, 4316, 1104, 1240, 1910, 117, 1105, 8698, 1828, 119, 19662, 1926, 1106, 1123, 119, 119, 102], [101, 1327, 1209, 1129, 1117, 3774, 117, 107, 1354, 1131, 117, 107, 1165, 1119, 3520, 1150, 1152, 1132, 106, 102], [101, 1337, 1119, 1431, 1256, 2936, 1106, 1123, 1108, 6929, 106, 102], [101, 1337, 1119, 1108, 3753, 1118, 1103, 14255, 24321, 1988, 1108, 10238, 132, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1,

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [128]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/307 [00:00<?, ? examples/s]

Map:   0%|          | 0/132 [00:00<?, ? examples/s]

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our tasks are about token classification, we use the `AutoModelForTokenClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem (which we can get from the features, as seen before):

In [129]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `Trainer`, we will need to define three more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [130]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=100,
    weight_decay=0.01,
    #push_to_hub=True,
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay.

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/bert-finetuned-ner"` or `"huggingface/bert-finetuned-ner"`).

Then we will need a data collator that will batch our processed examples together while applying padding to make them all the same size (each pad will be padded to the length of its longest example). There is a data collator for this task in the Transformers library, that not only pads the inputs, but also the labels:

In [131]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

The last thing to define for our `Trainer` is how to compute the metrics from the predictions. Here we will load the [`seqeval`](https://github.com/chakki-works/seqeval) metric (which is commonly used to evaluate results on the CONLL dataset) via the Datasets library.

In [132]:
metric = load_metric("seqeval")

This metric takes list of labels for the predictions and references:

In [133]:
labels = [label_list[i] for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:495: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


{'overall_precision': 0.0,
 'overall_recall': 0.0,
 'overall_f1': 0.0,
 'overall_accuracy': 1.0}

So we will need to do a bit of post-processing on our predictions:
- select the predicted index (with the maximum logit) for each token
- convert it to its string label
- ignore everywhere we set a label of -100

The following function does all this post-processing on the result of `Trainer.evaluate` (which is a namedtuple containing predictions and labels) before applying the metric:

In [134]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Note that we drop the precision/recall/f1 computed for each category and only focus on the overall precision/recall/f1/accuracy.

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [135]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

We can now finetune our model by just calling the `train` method:

In [136]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.064725,0.731707,0.722892,0.727273,0.982002
2,No log,0.052814,0.797619,0.807229,0.802395,0.986392
3,No log,0.053800,0.827160,0.807229,0.817073,0.987270
4,No log,0.052200,0.821429,0.831325,0.826347,0.988147
5,No log,0.058548,0.829268,0.819277,0.824242,0.987709
6,No log,0.059640,0.831325,0.831325,0.831325,0.988147
7,No log,0.059125,0.819277,0.819277,0.819277,0.987709
8,No log,0.060901,0.831325,0.831325,0.831325,0.988147
9,No log,0.061230,0.831325,0.831325,0.831325,0.988586
10,No log,0.052418,0.813953,0.843373,0.828402,0.988147


TrainOutput(global_step=2000, training_loss=0.0021883091730996967, metrics={'train_runtime': 341.699, 'train_samples_per_second': 89.845, 'train_steps_per_second': 5.853, 'total_flos': 572751273829986.0, 'train_loss': 0.0021883091730996967, 'epoch': 100.0})

The `evaluate` method allows you to evaluate again on the evaluation dataset or on another dataset:

In [137]:
trainer.evaluate()

{'eval_loss': 0.09327218681573868,
 'eval_precision': 0.8095238095238095,
 'eval_recall': 0.8192771084337349,
 'eval_f1': 0.8143712574850299,
 'eval_accuracy': 0.9877085162423178,
 'eval_runtime': 0.4422,
 'eval_samples_per_second': 298.5,
 'eval_steps_per_second': 20.352,
 'epoch': 100.0}

To get the precision/recall/f1 computed for each category now that we have finished training, we can apply the same function as before on the result of the `predict` method:

In [138]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'LOC': {'precision': 0.6666666666666666,
  'recall': 0.5714285714285714,
  'f1': 0.6153846153846153,
  'number': 14},
 'MISC': {'precision': 0.7,
  'recall': 0.6086956521739131,
  'f1': 0.6511627906976744,
  'number': 23},
 'PER': {'precision': 0.8846153846153846,
  'recall': 1.0,
  'f1': 0.9387755102040816,
  'number': 46},
 'overall_precision': 0.8095238095238095,
 'overall_recall': 0.8192771084337349,
 'overall_f1': 0.8143712574850299,
 'overall_accuracy': 0.9877085162423178}

You can now upload the result of the training to the Hub, just execute this instruction:

In [139]:
trainer.push_to_hub()

OSError: ignored

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained("sgugger/my-awesome-model")
```